Oskar Lundqvist; osauli-0@student.ltu.se

Filip Renberg; filren-0@student.ltu.se

We are gonna do a simple lab where we train a model using Keras

We are gonna use the mnist dataset of digits imported from keras like we did in lab 1 and lab 4.

In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow() # Remove non-important tensorflow warnings

import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
input_shape = (28,28,1)
num_classes = 10

We preprocess our data

In [ ]:
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
#normalize input values from RGB to Greyscale [0,1]
train_data = train_data.astype("float32")/255
test_data = test_data.astype("float32")/255
#Make sure images have the correct shape(28,28,1)
train_data = np.expand_dims(train_data, -1)
test_data = np.expand_dims(test_data, -1)

#converts class vectors to binary class matrices
train_labels = keras.utils.to_categorical(train_labels, num_classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes)

print("training shape: ", train_data.shape)
print(train_data.shape[0], "number of training samples")
print(test_data.shape[0], "number of testing samples")

We create our multi-layer neural network using Keras

In [ ]:
def create_model(nr_conv2d = 1, nr_filter = 32, kernel_size = (3, 3), model_summary = False):
    # Name the model based on the input values, 
    # model_nr_conv2d-nr_filter-kernel_size
    model_name=f"sequential_{nr_conv2d}-{nr_filter}-{kernel_size[0]}"
    
    hidden_layers = []

    hidden_layers.append(keras.Input(shape=input_shape)) # input layer

    for i in range(nr_conv2d):
        hidden_layers.append(layers.Conv2D(nr_filter, kernel_size=kernel_size, activation="relu"))
        hidden_layers.append(layers.MaxPooling2D(pool_size=[2,2]))

        nr_filter*=2

    hidden_layers.append(layers.Flatten())
    hidden_layers.append(layers.Dropout(0.5)) #prevents overfitting
    hidden_layers.append(layers.Dense(num_classes, activation="softmax"))

    model = keras.Sequential(hidden_layers, name=model_name)

    if model_summary == True:
        model.summary()
    else:
        print(f'Model: {model_name}')
    
    return model

Next we are training the model. We can use several different loss functions here, for this project we will use "categorical_crossentropy", "sparse_categorical_crossentropy" and "binary_crossentropy"

In [ ]:
n_epochs = 10
size_batch = 128

def model_training(pick, model):
    match pick:
        case 1:
            #categorical loss function
            model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
            print("Loss Function: categorical_crossentropy")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)
        case 2:
            #poisson loss function
            model.compile(loss="poisson", optimizer="adam", metrics=["accuracy"])
            print("Loss Function: poisson")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)
        case 3:
            #binary loss function
            model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
            print("Loss Function: binary_crossentropy")
            model.fit(train_data, train_labels, batch_size=size_batch, epochs=n_epochs, validation_split=0.1, verbose=0)


Define for the test diffrent values we want to test

In [ ]:
nr_conv2d_list = [1, 2, 3]
nr_filter_list = [16, 32, 64]

# We use this list to save the values for the best model
best_model = [[0, 1]]

In [ ]:
def best_performance(best_model):
    # Resulting best model:
    create_model(nr_conv2d=best_model[0][2], nr_filter=best_model[0][3])
    match best_model[0][4]:
        case 1:
            print("Using Categorical crossentropy loss function it gave us the \nfollowing accuracy and loss:")
        case 2:
            print("Using Poisson loss function it gave us the \nfollowing accuracy and loss:")
        case 3:
            print("Using binary crossentropy loss function it gave us the \nfollowing accuracy and loss")
    print("|\taccuracy: \t", best_model[0][0], "\t|")
    print("|\tloss: \t\t", best_model[0][1], "\t|")

In [ ]:
print("We mute model summary for the test prints but you can still see the model name:")
print("first value is the nr of conv2d layers, second value is the starting filter size.")
print("The fird value is the kernel size but we don't change it in our tests.\n")

for i in nr_conv2d_list: # The nr conv2d layers
    for j in nr_filter_list: # The starting size of the conv2d filters
        for n in range(3):

            model = create_model(nr_conv2d=i, nr_filter=j, kernel_size=(3, 3)) # You can also change kernel size
            train = model_training(n+1, model)

            #evaluate
            score = model.evaluate(train_data, train_labels, verbose=0)
            print("|\taccuracy: \t", score[1], "\t|")
            print("|\tloss: \t\t", score[0], "\t|")

            if (score[1] > best_model[0][0]) & (score[0] < best_model[0][1]):
                best_model.pop()
                best_model.append((score[1], score[0], i, j, n+1))

            print("\n")

print("Finally done :)")

The following is the best performing model from our tests:

In [ ]:
best_performance(best_model)